In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))

In [2]:
import pandas as pd

from mcda.electre.discordance import discordance_bin, discordance_bin_comprehensive, discordance_bin_marginal
from mcda.core.scales import QuantitativeScale, PreferenceDirection
from mcda.core.functions import Threshold

1. Create dataframes

In [3]:
alternative_names = ['fiat', 'vwPolo' , 'nissan' , 'toyota' , 'suzuki', 'ford' ]
criteria_names = ['C1', 'C2', 'C3']
profile_names = ['P1', 'P2', 'P3']

In [4]:
fiat = [100, 1000, 6.0]
vwPolo = [150, 5000, 5.7]
nissan = [110, 4000, 6.8]
toyota = [170, 3000, 5.5]
suzuki = [140, 2000, 7.8]
ford = [120, 6000, 8.0]
cars = pd.DataFrame(
    [fiat, vwPolo, nissan, toyota, suzuki, ford],
    index=alternative_names,
    columns=criteria_names,
)
cars

,C1,C2,C3
fiat,100,1000,6.0
vwPolo,150,5000,5.7
nissan,110,4000,6.8
toyota,170,3000,5.5
suzuki,140,2000,7.8
ford,120,6000,8.0


In [5]:
profile1 = [113, 1200, 7.0]
profile2 = [180, 3500, 6.7]
profile3 = [199, 3400, 8.8]
profiles = pd.DataFrame(
    [profile1, profile2, profile3],
    index=profile_names,
    columns=criteria_names
) 
profiles

,C1,C2,C3
P1,113,1200,7.0
P2,180,3500,6.7
P3,199,3400,8.8


2. Create criteria scales and veto thresholds

In [6]:
scales = pd.Series([
    QuantitativeScale(100, 500),
    QuantitativeScale(1000, 6000, PreferenceDirection.MIN),
    QuantitativeScale(0.0, 15.0, PreferenceDirection.MIN)
], index=criteria_names)
scales

C1      Scale [100, 500], max direction
C2    Scale [1000, 6000], min direction
C3     Scale [0.0, 15.0], min direction
dtype: object

In [7]:
veto = pd.Series([
    Threshold(0, 40),
    Threshold(0, 500),
    Threshold(0.5, 0.1)
], index=criteria_names)
veto

C1       Threshold: alpha=0, beta=40
C2      Threshold: alpha=0, beta=500
C3    Threshold: alpha=0.5, beta=0.1
dtype: object

3. Let's begin with some marginal concordance calculations :)

Marginal discordance index between fiat and suzuki on the C2 criterion:

In [8]:
discordance_bin_marginal(
    cars['C2']['fiat'],
    cars['C2']['suzuki'],
    scales['C2'],
    veto['C2']
)

0

4. Short example with comprehensive index calculations

Again, between fiat and suzuki:

In [9]:
discordance_bin_comprehensive(
    cars.loc['fiat'],
    cars.loc['suzuki'],
    scales,
    veto
)

1

5. Concordance table between alternatives and alternatives

In [10]:
discordance_bin(
    cars,
    scales,
    veto
)

,fiat,vwPolo,nissan,toyota,suzuki,ford
fiat,0,1,0,1,1,0
vwPolo,1,0,1,1,1,0
nissan,1,1,0,1,1,0
toyota,1,0,0,0,1,0
suzuki,1,0,0,0,0,0
ford,1,1,1,1,1,0


6. Concordance table between alternatives and profiles

In [11]:
alt_prof, prof_alt = discordance_bin(
    cars,
    scales,
    veto,
    profiles,
)

In [12]:
alt_prof

,P1,P2,P3
fiat,0,1,1
vwPolo,1,1,1
nissan,1,1,1
toyota,1,0,0
suzuki,1,1,1
ford,1,1,1


In [13]:
prof_alt

,fiat,vwPolo,nissan,toyota,suzuki,ford
P1,0,0,0,1,0,0
P2,1,0,0,1,1,0
P3,1,0,0,0,1,0
